##**2. Word2Vec**
1. 주어진 단어들을 word2vec 모델에 들어갈 수 있는 형태로 만듭니다.
2. CBOW, Skip-gram 모델을 각각 구현합니다.
3. 모델을 실제로 학습해보고 결과를 확인합니다.

### **필요 패키지 import**

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 50.5MB/s 
     |████████████████████████████████| 92kB 13.1MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
from tqdm import tqdm
from konlpy.tag import Okt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

import torch
import copy
import numpy as np

### **데이터 전처리**



데이터를 확인하고 Word2Vec 형식에 맞게 전처리합니다.  
학습 데이터는 1번 실습과 동일하고, 테스트를 위한 단어를 아래와 같이 가정해봅시다.

In [3]:
train_data = [
  "정말 맛있습니다. 추천합니다.",
  "기대했던 것보단 별로였네요.",
  "다 좋은데 가격이 너무 비싸서 다시 가고 싶다는 생각이 안 드네요.",
  "완전 최고입니다! 재방문 의사 있습니다.",
  "음식도 서비스도 다 만족스러웠습니다.",
  "위생 상태가 좀 별로였습니다. 좀 더 개선되기를 바랍니다.",
  "맛도 좋았고 직원분들 서비스도 너무 친절했습니다.",
  "기념일에 방문했는데 음식도 분위기도 서비스도 다 좋았습니다.",
  "전반적으로 음식이 너무 짰습니다. 저는 별로였네요.",
  "위생에 조금 더 신경 썼으면 좋겠습니다. 조금 불쾌했습니다."       
]

test_words = ["음식", "맛", "서비스", "위생", "가격"]

Tokenization과 vocab을 만드는 과정은 이전 실습과 유사합니다.

In [4]:
tokenizer = Okt()

In [5]:
def make_tokenized(data):
  tokenized = []
  for sent in tqdm(data):
    tokens = tokenizer.morphs(sent, stem=True)
    tokenized.append(tokens)

  return tokenized

In [6]:
train_tokenized = make_tokenized(train_data)

100%|██████████| 10/10 [00:05<00:00,  1.83it/s]


In [7]:
word_count = defaultdict(int)

for tokens in tqdm(train_tokenized):
  for token in tokens:
    word_count[token] += 1

100%|██████████| 10/10 [00:00<00:00, 22168.63it/s]


In [8]:
word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
print(list(word_count))

[('.', 14), ('도', 7), ('이다', 4), ('좋다', 4), ('별로', 3), ('다', 3), ('이', 3), ('너무', 3), ('음식', 3), ('서비스', 3), ('하다', 2), ('방문', 2), ('위생', 2), ('좀', 2), ('더', 2), ('에', 2), ('조금', 2), ('정말', 1), ('맛있다', 1), ('추천', 1), ('기대하다', 1), ('것', 1), ('보단', 1), ('가격', 1), ('비싸다', 1), ('다시', 1), ('가다', 1), ('싶다', 1), ('생각', 1), ('안', 1), ('드네', 1), ('요', 1), ('완전', 1), ('최고', 1), ('!', 1), ('재', 1), ('의사', 1), ('있다', 1), ('만족스럽다', 1), ('상태', 1), ('가', 1), ('개선', 1), ('되다', 1), ('기르다', 1), ('바라다', 1), ('맛', 1), ('직원', 1), ('분들', 1), ('친절하다', 1), ('기념일', 1), ('분위기', 1), ('전반', 1), ('적', 1), ('으로', 1), ('짜다', 1), ('저', 1), ('는', 1), ('신경', 1), ('써다', 1), ('불쾌하다', 1)]


In [9]:
w2i = {}
for pair in tqdm(word_count):
  if pair[0] not in w2i:
    w2i[pair[0]] = len(w2i)

100%|██████████| 60/60 [00:00<00:00, 221140.81it/s]


In [10]:
print(train_tokenized)
print(w2i)

[['정말', '맛있다', '.', '추천', '하다', '.'], ['기대하다', '것', '보단', '별로', '이다', '.'], ['다', '좋다', '가격', '이', '너무', '비싸다', '다시', '가다', '싶다', '생각', '이', '안', '드네', '요', '.'], ['완전', '최고', '이다', '!', '재', '방문', '의사', '있다', '.'], ['음식', '도', '서비스', '도', '다', '만족스럽다', '.'], ['위생', '상태', '가', '좀', '별로', '이다', '.', '좀', '더', '개선', '되다', '기르다', '바라다', '.'], ['맛', '도', '좋다', '직원', '분들', '서비스', '도', '너무', '친절하다', '.'], ['기념일', '에', '방문', '하다', '음식', '도', '분위기', '도', '서비스', '도', '다', '좋다', '.'], ['전반', '적', '으로', '음식', '이', '너무', '짜다', '.', '저', '는', '별로', '이다', '.'], ['위생', '에', '조금', '더', '신경', '써다', '좋다', '.', '조금', '불쾌하다', '.']]
{'.': 0, '도': 1, '이다': 2, '좋다': 3, '별로': 4, '다': 5, '이': 6, '너무': 7, '음식': 8, '서비스': 9, '하다': 10, '방문': 11, '위생': 12, '좀': 13, '더': 14, '에': 15, '조금': 16, '정말': 17, '맛있다': 18, '추천': 19, '기대하다': 20, '것': 21, '보단': 22, '가격': 23, '비싸다': 24, '다시': 25, '가다': 26, '싶다': 27, '생각': 28, '안': 29, '드네': 30, '요': 31, '완전': 32, '최고': 33, '!': 34, '재': 35, '의사': 36, '있다': 37, '만족스럽다': 38, '상태

실제 모델에 들어가기 위한 input을 만들기 위해 `Dataset` 클래스를 정의합니다.

In [11]:
class CBOWDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.x.append(token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.y.append(id)

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수, 2 * window_size)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [12]:
class SkipGramDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.y += (token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.x += [id] * 2 * window_size

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

각 모델에 맞는 `Dataset` 객체를 생성합니다.

In [13]:
cbow_set = CBOWDataset(train_tokenized)
skipgram_set = SkipGramDataset(train_tokenized)
print(list(skipgram_set))

100%|██████████| 10/10 [00:00<00:00, 5658.80it/s]

[(tensor(0), tensor(17)), (tensor(0), tensor(18)), (tensor(0), tensor(19)), (tensor(0), tensor(10)), (tensor(19), tensor(18)), (tensor(19), tensor(0)), (tensor(19), tensor(10)), (tensor(19), tensor(0)), (tensor(22), tensor(20)), (tensor(22), tensor(21)), (tensor(22), tensor(4)), (tensor(22), tensor(2)), (tensor(4), tensor(21)), (tensor(4), tensor(22)), (tensor(4), tensor(2)), (tensor(4), tensor(0)), (tensor(23), tensor(5)), (tensor(23), tensor(3)), (tensor(23), tensor(6)), (tensor(23), tensor(7)), (tensor(6), tensor(3)), (tensor(6), tensor(23)), (tensor(6), tensor(7)), (tensor(6), tensor(24)), (tensor(7), tensor(23)), (tensor(7), tensor(6)), (tensor(7), tensor(24)), (tensor(7), tensor(25)), (tensor(24), tensor(6)), (tensor(24), tensor(7)), (tensor(24), tensor(25)), (tensor(24), tensor(26)), (tensor(25), tensor(7)), (tensor(25), tensor(24)), (tensor(25), tensor(26)), (tensor(25), tensor(27)), (tensor(26), tensor(24)), (tensor(26), tensor(25)), (tensor(26), tensor(27)), (tensor(26), tens

### **모델 Class 구현**

차례대로 두 가지 Word2Vec 모델을 구현합니다.  


*   `self.embedding`: `vocab_size` 크기의 one-hot vector를 특정 크기의 `dim` 차원으로 embedding 시키는 layer.
*   `self.linear`: 변환된 embedding vector를 다시 원래 `vocab_size`로 바꾸는 layer.


In [14]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, dim):
    super(CBOW, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x):  # x: (B, 2W)
    embeddings = self.embedding(x)  # (B, 2W, d_w)
    embeddings = torch.sum(embeddings, dim=1)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

In [15]:
class SkipGram(nn.Module):
  def __init__(self, vocab_size, dim):
    super(SkipGram, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x): # x: (B)
    embeddings = self.embedding(x)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

두 가지 모델을 생성합니다.

In [16]:
cbow = CBOW(vocab_size=len(w2i), dim=256)
skipgram = SkipGram(vocab_size=len(w2i), dim=256)

### **모델 학습**

다음과 같이 hyperparamter를 세팅하고 `DataLoader` 객체를 만듭니다.

In [17]:
batch_size=4
learning_rate = 5e-4
num_epochs = 5
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

cbow_loader = DataLoader(cbow_set, batch_size=batch_size)
skipgram_loader = DataLoader(skipgram_set, batch_size=batch_size)

첫번째로 CBOW 모델 학습입니다.

In [18]:
cbow.train()
cbow = cbow.to(device)
optim = torch.optim.SGD(cbow.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(cbow_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = cbow(x)  # (B, V)
 
    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  0%|          | 0/16 [00:00<?, ?it/s]

##################################################
Epoch: 1
Train loss: 4.918984413146973
Train loss: 4.808871269226074
Train loss: 4.287832260131836
Train loss: 5.530417442321777
Train loss: 5.20170259475708
Train loss: 3.826876640319824
Train loss: 5.974105358123779
Train loss: 4.024430274963379
Train loss: 4.936190605163574
Train loss: 4.39900016784668
Train loss: 4.367701053619385
Train loss: 4.321840286254883
Train loss: 4.843523025512695
Train loss: 4.647263050079346
Train loss: 4.8774542808532715
Train loss: 6.048351764678955
##################################################
Epoch: 2
Train loss: 4.783356666564941
Train loss: 4.6636433601379395


100%|██████████| 16/16 [00:00<00:00, 635.69it/s]

Train loss: 4.155683517456055
Train loss: 5.379117012023926
Train loss: 5.057653427124023
Train loss: 3.5813560485839844
Train loss: 5.766635894775391
Train loss: 3.8913605213165283
Train loss: 4.785010814666748
Train loss: 4.226993560791016
Train loss: 4.186971187591553
Train loss: 3.970266819000244
Train loss: 4.685272216796875
Train loss: 4.523494720458984
Train loss: 4.709317207336426
Train loss: 5.884603977203369
##################################################
Epoch: 3
Train loss: 4.648874759674072
Train loss: 4.520533084869385
Train loss: 4.025557994842529
Train loss: 5.229218006134033
Train loss: 4.915145397186279
Train loss: 3.352673053741455
Train loss: 5.563316345214844
Train loss: 3.761737108230591
Train loss: 4.6386399269104
Train loss: 4.061951637268066
Train loss: 4.0133538246154785
Train loss: 3.635598659515381
Train loss: 4.529045104980469
Train loss: 4.4024271965026855
Train loss: 4.543397903442383
Train loss: 5.724033355712891
######################################

다음으로 Skip-gram 모델 학습입니다.

In [19]:
skipgram.train()
skipgram = skipgram.to(device)
optim = torch.optim.SGD(skipgram.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(skipgram_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = skipgram(x)  # (B, V)

    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  0%|          | 0/64 [00:00<?, ?it/s]

##################################################
Epoch: 1
Train loss: 4.207857131958008
Train loss: 3.5234036445617676
Train loss: 4.5736188888549805
Train loss: 4.267858982086182
Train loss: 4.011657238006592
Train loss: 4.458869934082031
Train loss: 4.906720161437988
Train loss: 3.988703727722168
Train loss: 4.161901473999023
Train loss: 4.5914082527160645
Train loss: 4.794273376464844
Train loss: 4.027994155883789
Train loss: 4.529797554016113
Train loss: 4.295514106750488
Train loss: 4.459736347198486
Train loss: 4.531698226928711
Train loss: 4.458432197570801
Train loss: 4.799626350402832
Train loss: 4.328784942626953
Train loss: 4.442192554473877
Train loss: 3.901571035385132
Train loss: 4.215538024902344
Train loss: 4.187392711639404
Train loss: 4.00093936920166
Train loss: 4.009987831115723
Train loss: 4.1357035636901855
Train loss: 4.453004837036133
Train loss: 4.223850727081299
Train loss: 3.6689605712890625
Train loss: 4.510236740112305
Train loss: 3.7955877780914307
Train

  0%|          | 0/64 [00:00<?, ?it/s]

Train loss: 4.327714920043945
Train loss: 3.8385539054870605
Train loss: 4.003208160400391
Train loss: 4.287541389465332
Train loss: 4.339561462402344
Train loss: 4.400043487548828
Train loss: 4.139243125915527
Train loss: 4.439983367919922
Train loss: 4.110322952270508
Train loss: 3.3992233276367188
##################################################
Epoch: 2
Train loss: 4.182790279388428
Train loss: 3.478567361831665
Train loss: 4.5398054122924805
Train loss: 4.209433555603027
Train loss: 3.9777188301086426
Train loss: 4.418764591217041
Train loss: 4.859943866729736
Train loss: 3.9627890586853027
Train loss: 4.135342597961426
Train loss: 4.563461780548096
Train loss: 4.758031368255615
Train loss: 3.9936928749084473
Train loss: 4.5003252029418945
Train loss: 4.266109466552734
Train loss: 4.421187400817871
Train loss: 4.502606391906738
Train loss: 4.43149471282959
Train loss: 4.761442184448242
Train loss: 4.299889087677002
Train loss: 4.410280227661133
Train loss: 3.815575122833252
Trai

  0%|          | 0/64 [00:00<?, ?it/s]

Train loss: 3.9440221786499023
Train loss: 4.37922477722168
Train loss: 4.813733100891113
Train loss: 3.9370288848876953
Train loss: 4.108903408050537
Train loss: 4.53563117980957
Train loss: 4.721955299377441
Train loss: 3.9596235752105713
Train loss: 4.471277236938477
Train loss: 4.236855506896973
Train loss: 4.382852077484131
Train loss: 4.473780155181885
Train loss: 4.404668807983398
Train loss: 4.723435401916504
Train loss: 4.271284103393555
Train loss: 4.378574371337891
Train loss: 3.731248617172241
Train loss: 4.041170597076416
Train loss: 4.101564407348633
Train loss: 3.943929433822632
Train loss: 3.9434947967529297
Train loss: 4.02498197555542
Train loss: 4.363460540771484
Train loss: 4.165288925170898
Train loss: 3.6091456413269043
Train loss: 4.446080207824707
Train loss: 3.7431960105895996
Train loss: 4.406737327575684
Train loss: 4.418460369110107
Train loss: 4.160217761993408
Train loss: 4.092319011688232
Train loss: 3.856858968734741
Train loss: 3.767615795135498
Train l

  0%|          | 0/64 [00:00<?, ?it/s]

Train loss: 4.134123802185059
Train loss: 3.3902945518493652
Train loss: 4.472687721252441
Train loss: 4.094958305358887
Train loss: 3.9105706214904785
Train loss: 4.340254306793213
Train loss: 4.768090724945068
Train loss: 3.911424398422241
Train loss: 4.082584381103516
Train loss: 4.50791597366333
Train loss: 4.686046600341797
Train loss: 3.925790786743164
Train loss: 4.442651748657227
Train loss: 4.2077531814575195
Train loss: 4.3447346687316895
Train loss: 4.445223808288574
Train loss: 4.377955436706543
Train loss: 4.685609340667725
Train loss: 4.242969989776611
Train loss: 4.347077369689941
Train loss: 3.6487274169921875
Train loss: 3.9576616287231445
Train loss: 4.059053421020508
Train loss: 3.9156739711761475
Train loss: 3.9109838008880615
Train loss: 3.970818519592285
Train loss: 4.319183349609375
Train loss: 4.136853218078613
Train loss: 3.579991102218628
Train loss: 4.414514541625977
Train loss: 3.7172529697418213
Train loss: 4.378304481506348
Train loss: 4.394967555999756
Tr

100%|██████████| 64/64 [00:00<00:00, 749.37it/s]

Train loss: 4.016809463500977
Train loss: 3.8875880241394043
Train loss: 3.8789727687835693
Train loss: 3.9175055027008057
Train loss: 4.275241851806641
Train loss: 4.108973503112793
Train loss: 3.551347494125366
Train loss: 4.383285045623779
Train loss: 3.691479206085205
Train loss: 4.350000381469727
Train loss: 4.371538162231445
Train loss: 4.102349281311035
Train loss: 4.037790298461914
Train loss: 3.801474094390869
Train loss: 3.674910545349121
Train loss: 3.7314136028289795
Train loss: 3.946455717086792
Train loss: 4.8720903396606445
Train loss: 4.311922073364258
Train loss: 4.233147621154785
Train loss: 4.053537368774414
Train loss: 3.709861993789673
Train loss: 3.366763114929199
Train loss: 3.6742324829101562
Train loss: 3.5340158939361572
Train loss: 4.1080780029296875
Train loss: 4.147226810455322
Train loss: 3.8367271423339844
Train loss: 4.355772018432617
Train loss: 4.005873203277588
Train loss: 4.391183376312256
Train loss: 4.175941467285156
Train loss: 4.1996564865112305


### **테스트**

학습된 각 모델을 이용하여 test 단어들의 word embedding을 확인합니다.

In [20]:
for word in test_words:
  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = cbow.embedding(input_id)

  print(f"Word: {word}")
  print(emb.squeeze(0))

Word: 음식
tensor([-0.3703,  0.2070, -0.0210, -0.4419, -0.6693,  0.6081, -1.0257, -1.6998,
         0.0335, -1.1137,  0.2362,  1.8460,  2.6074, -0.0270, -1.1597, -0.3125,
         0.4015, -0.5558,  1.0377,  1.2693, -0.3645,  2.2569,  0.4265, -1.2970,
        -0.7456,  1.5841,  0.0761,  0.1560, -0.0097,  0.1958,  1.8457, -1.1822,
        -0.0048, -0.5153,  1.1439,  0.3196, -0.6785,  1.2522, -0.9340,  0.8138,
         0.0424, -0.4504, -0.3001,  0.0934, -0.0923,  0.3502, -0.1752, -0.2143,
         0.4093, -1.4581, -0.3644,  0.3045,  0.3902,  0.4472,  0.9183, -0.6240,
        -0.3673, -0.6556,  1.9662, -0.2060, -0.6740,  1.6264, -0.2121, -0.0408,
        -0.0101,  0.2785, -0.1552, -1.3472, -0.6049,  0.1696, -0.5029,  1.2507,
        -1.0286,  0.6753,  1.2800, -1.2351, -0.2297, -0.8465, -0.1557, -0.1328,
         1.1539, -0.9876, -0.7554, -0.4650, -0.9811,  1.9723, -0.0950, -0.5834,
         1.0778,  1.5228, -0.5098, -0.1756, -1.0877, -0.7893,  1.1786,  0.2906,
        -0.6684, -0.7622,  0.29

In [21]:
for word in test_words:
  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = skipgram.embedding(input_id)

  print(f"Word: {word}")
  print(emb.squeeze(0))

Word: 음식
tensor([ 1.8570e+00, -1.4394e+00, -1.1833e+00, -2.5421e+00, -1.9595e+00,
        -3.5944e-01, -3.4530e-01, -1.1860e+00,  2.4947e-01, -6.1721e-01,
         3.8400e-01,  5.0110e-01, -1.2098e+00, -2.9716e-01,  7.6627e-01,
        -8.3425e-02,  8.4570e-01,  9.5548e-01,  2.1861e-01,  3.0166e-01,
         1.2819e+00, -5.2614e-01,  1.1436e+00,  5.4008e-02, -7.4878e-01,
         3.5178e-01,  1.2302e+00,  5.7593e-01,  1.9396e+00, -1.6141e+00,
        -9.3632e-01, -1.2891e-01, -1.7853e+00, -1.1406e+00, -3.2449e+00,
        -2.6642e+00,  1.5147e-01,  1.2784e+00, -3.8765e-01,  1.2116e-02,
        -4.0217e-01,  8.0555e-01,  4.1635e-01, -5.0729e-01,  1.7464e-02,
         3.2053e+00,  6.6062e-01, -8.1058e-01, -1.3059e+00,  8.8530e-01,
         3.3934e-01,  1.3479e+00, -3.7349e-01, -2.1716e-01, -1.9377e-01,
        -1.8388e+00, -4.5450e-01,  3.8593e-01, -1.9482e+00,  8.4301e-01,
         8.9054e-01, -1.6179e+00, -5.3505e-01, -1.7304e-01, -6.8485e-03,
        -5.8680e-01, -2.4607e-01,  3.9138e